In [ ]:
#default_exp structure

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Structure

> Classes to create hierachy configurations.

In [ ]:
#hide
import sys
sys.path.append("..")

In [ ]:
#export
import enum
import uuid
from pct.functions import ControlUnitFunctions
from pct.functions import WeightedSum

In [ ]:
#export
class LevelKey(enum.Enum):
   ZERO = 'level0'
   N = 'leveln'
   TOP = 'leveltop'
   ZEROTOP = 'level0top'

In [ ]:
#export
class ArchitectureStructure():
    "ArchitectureStructure"
    def __init__(self, references=None, config=None, attr_mut_pb=None, lower_float=None, upper_float=None, levels_limit=None, 
                 columns_limit=None, sigma=None, mu=None, alpha=None, modes=None, namespace=None,**cargs):
        
        if namespace ==None:
            namespace = uuid.uuid1()
        self.namespace=namespace
        
        if config==None:
            self.config={'parameters': { 'modes' : {LevelKey.ZERO:3, LevelKey.N:3,LevelKey.TOP:4,LevelKey.ZEROTOP :4} }}                 
        else:
            self.config=config
        
        if modes!=None:  
            self.config['parameters']['modes']=modes


    def get_config(self):
        return self.config

    def set_node_function(self, node, function, mode, thislevel, targetlevel, targetprefix, targetcolumn, 
                          num_target_indices, inputs, input_weights, by_column, offset):
        
        func = node.get_function_from_collection(function)
        func.set_node_function(function, thislevel, targetlevel, targetprefix, targetcolumn, num_target_indices, inputs, 
                               input_weights, by_column, offset)
       
        
        #type = PCTNode.get_function_type(mode, function)
        #function_type = FunctionFactory.createFunction(type)
        #function_type.set_node_function(node, function,  thislevel, targetlevel, targetprefix, targetcolumn, num_target_indices, inputs, input_weights, by_column)
        
        
        
        
    def set_sparse_node_function(self, node, function, thislevel, input, column, input_weights):
        func = node.get_function_from_collection(function)
        func.set_sparse_node_function(function, thislevel, input, column, input_weights)

    def set_namespace(self,namespace):
        self.namespace=namespace

        
    def get_parameter(self, key):
        return self.config['parameters'][key]

    
    def set_output_function(self, node, mode, thislevel, column, input_weights):
        func = node.get_function_from_collection(ControlUnitFunctions.OUTPUT)
        func.set_output_function(thislevel, column, input_weights)

        #type = PCTNode.get_function_type(mode, ControlUnitFunctions.OUTPUT)
        #function_type = FunctionFactory.createFunction(type)
        #function_type.set_output_function(node, thislevel, column, input_weights)

    
    def set_action_function(self, hpct, env, numColumnsThisLevel,  weights):
        numActions = len(weights)
        for actionIndex in range(numActions):
            action = WeightedSum(weights=weights[actionIndex], name=f'Action{actionIndex+1}', namespace=self.namespace)
            for column in range(numColumnsThisLevel):
                action.add_link(f'OL0C{column}')
            hpct.add_postprocessor(action)
            env.add_link(action)

In [ ]:
arch = ArchitectureStructure()

In [ ]:
config = arch.get_config()
print(arch.get_config())
target =  {'parameters': { 'modes' : {LevelKey.ZERO:3, LevelKey.N:3,LevelKey.TOP:4,LevelKey.ZEROTOP :4} }}
assert config == target

{'parameters': {'modes': {<LevelKey.ZERO: 'level0'>: 3, <LevelKey.N: 'leveln'>: 3, <LevelKey.TOP: 'leveltop'>: 4, <LevelKey.ZEROTOP: 'level0top'>: 4}}}
